In [ ]:
import numpy as np
import pandas as pd

# Settings

In [ ]:
settings = { 
            'data_size': None, # size of data subset after shuffle is performed
            'test_size': 0.25, # fraction of data set to be assigned as test data
            'save_env': False, # save environment
            'del': True, # Delete variables that are no longer needed to proceed in computations to save place
            'learning_curve': { # parameters to generate learning_curve
                'start': int(5e4), # training set start size (including)
                'stop': int(1e6 + 2.5e4), # traing set stop size (excluding)
                'step': int(2.5e4)  # increase between iterations
#                 'start': int(5e2), # training set start size (including)
#                 'stop': int(1e4 + 2.5e2), # traing set stop size (excluding)
#                 'step': int(2.5e2)  # increase between iterations
                              }
           }

# Load Data: load dataset, shuffle it and take subset

In [ ]:
PATH = "../data/"

In [ ]:
data_full = pd.read_csv(f'{PATH}train.tsv', sep='\t')

In [ ]:
data_full.head()

In [ ]:
len(data_full)

In [ ]:
from sklearn.utils import shuffle
data_shuffled = shuffle(data_full)

In [ ]:
if (settings['del']):
    del data_full

In [ ]:
data = data_shuffled.iloc[:settings['data_size'], :]

In [ ]:
if (settings['del']):
    del data_shuffled

In [ ]:
len(data)

In [ ]:
data.head()

# Data preparation

## Item Category

Split category_name into main_cat, subcat_1 and subcat_2

In [ ]:
# reference: BuryBuryZymon at https://www.kaggle.com/maheshdadhich/i-will-sell-everything-for-free-0-55
def split_cat(text):
    try: return text.split("/")
    except: return ("No Label", "No Label", "No Label")

In [ ]:
data.loc[:,'main_cat'], data.loc[:,'subcat_1'], data.loc[:,'subcat_2'] = \
zip(*data.loc[:,'category_name'].apply(lambda x: split_cat(x)))
# data.head()

In [ ]:
data.head()

## Overview

In [ ]:
print("Number of unique fields:\n")

print("main_cat: \t%d" % data['main_cat'].nunique())
print("subcat_1: \t%d" % data['subcat_1'].nunique())
print("subcat_2: \t%d" % data['subcat_2'].nunique())
print("brand_name: \t%d" % data['brand_name'].nunique())
print()

print("%d items have no category" % len(data.loc[data['main_cat'] == 'No Label']))
print("%d items have no brand" % data['brand_name'].isna().sum())

## Numerically represent features

### train_id: copy

Create new DataFrame called data_num for numerical representations

In [ ]:
data_num = pd.DataFrame(data.loc[:,'train_id'], columns=['train_id'])

### name: represent name by its length as name_len

In [ ]:
data_num.loc[:,'name_len'] = data['name'].str.len()

### item_condition_id, price, shipping: copy

In [ ]:
data_num[['item_condition_id', 'price', 'shipping']] = data.loc[:,['item_condition_id', 'price', 'shipping']]

### item_description

Represent item_description by its length as item_description_len

In [ ]:
item_description_len = data.loc[:,'item_description'].str.len()
data_num['item_description_len'] = item_description_len

# Replace NaN in data_num.item_description_len column by zeros
data_num['item_description_len'] = data_num['item_description_len'].fillna(0)

# Change data type of this column to uint16 provided the max val is less than 65535
if (data_num.item_description_len.max() < 65535):
    data_num['item_description_len'] = data_num['item_description_len'].astype(np.uint16)

In [ ]:
# data_num.head()

In [ ]:
# data.head()

### Define function for making binary columns

In [ ]:
def make_binary_columns(df_str, df_num, column_name):
    """
    Turns a single column named column_name (with various categories) into m binary columns, where m is the number
    of unique values in the original column. For each sample, the value for all new columns is 0 apart the one
    that matches the value of the original column. Names of new binary columns are formed as follows:
    column_name + '_' + str(original column value)
    Inputs:
        two pandas DataFrames: df_str where a single column contains information about given category
                               df_num that will later contain many binary columns
        column_name: name of the column that will be split into several binary columns
    Returns nothing. It appends the collumns directly into df_num to increase performance.
    """
    m = len(df_str)
    if (m != len(df_num)):
        raise Exception("df_str and df_num must have the same size.")
 
    categories = df_str[column_name].unique()

    from tqdm import tqdm_notebook # progress bar

    # Create a new feature for each category and initialize it to 0
    for i in tqdm_notebook(categories, desc='1/2'):
        df_num[column_name + '_' + str(i)] = np.zeros((m, 1), dtype=np.int8)

#     Loop thorugh all rows and assign 1 to the column whose name is the same as category
    for i in tqdm_notebook(df_str.index, desc='2/2'): # loop through all rows
        category = str(df_str.at[i, column_name])
        df_num.at[i, column_name + '_' + category] = 1

### brand_name: for each unique one create new binary feature

In [ ]:
make_binary_columns(data, data_num, 'brand_name')

In [ ]:
# data.head()

### main_cat, subcat_1, subcat_2: for each unique one create new binary feature

In [ ]:
make_binary_columns(data, data_num, 'main_cat')
make_binary_columns(data, data_num, 'subcat_1')
make_binary_columns(data, data_num, 'subcat_2')

In [ ]:
if (settings['del']):
    del data

# Split data and extract X, y and train_id

In [ ]:
from sklearn.model_selection import train_test_split
X_train_unscaled, X_test_unscaled = train_test_split(data_num, test_size = settings['test_size'], random_state=None) # randomly split data
# ! X_train_unscaled and X_test_unscaled STILL CONTAINS PRICE AT THIS MOMENT !

In [ ]:
if (settings['del']):
    del data_num

In [ ]:
# Pop price from X_train and X_test
y_train = X_train_unscaled.pop('price')
y_test = X_test_unscaled.pop('price')

In [ ]:
# Pop id_train from both training and test data set

id_train = X_train_unscaled.pop('train_id')
id_test = X_test_unscaled.pop('train_id')

# Scale data

In [ ]:
from sklearn.preprocessing import MaxAbsScaler

X_train = X_train_unscaled.copy()
X_test = X_test_unscaled.copy()

columns_to_scale = ['name_len', 'item_condition_id', 'item_description_len']

scaler = MaxAbsScaler()
scaler.fit(X_train_unscaled[columns_to_scale]) # Compute the mean adn std of training data to be used for future scaling

X_train[columns_to_scale] = pd.DataFrame(scaler.transform(X_train_unscaled[columns_to_scale]), index=X_train_unscaled.index, columns=columns_to_scale)
if (settings['del']):
    del X_train_unscaled

X_test[columns_to_scale] = pd.DataFrame(scaler.transform(X_test_unscaled[columns_to_scale]), index=X_test_unscaled.index, columns=columns_to_scale)
if (settings['del']):
    del X_test_unscaled

In [ ]:
# X_train_unscaled.info()

In [ ]:
X_train.info()

In [ ]:
X_train.head()

In [ ]:
# X_test_unscaled.info()

In [ ]:
X_test.info()

In [ ]:
# X_train.to_csv('X_train.csv')
# X_train_scaled.to_csv('X_train_scaled.csv')
# X_test.to_csv('X_test.csv')
# X_test_scaled.to_csv('X_test_scaled.csv')

In [ ]:
if (settings['save_env']):
    import dill                            #pip install dill --user
    dill.dump_session('splittedData.pkl')

# Check size of variables

In [ ]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' By Fred Cirera, after https://stackoverflow.com/a/1094933/1870254'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name,value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name,sizeof_fmt(size)))

# Performance measure: RMSE

$$\text{RMSE} \left( \mathbf{Y} , \mathbf{\hat{Y}} \right) = \sqrt{\frac{1}{n} \sum_{i=1}^n \left( y_i - \hat{y_i} \right)^2 } $$

In [ ]:
def rmse(y_test, y_pred):
    from sklearn.metrics import mean_squared_error
    return np.sqrt(mean_squared_error(y_test, y_pred))

# Learning curve for linear regression

In [ ]:
from tqdm import tqdm_notebook # progress bar

# Initialize arrays for plotting learning curves
training_set_sizes = []
train_rmses = []
test_rmses = []

# Create file with headings to save training and test RMSEs for learning curves
import datetime
filename = datetime.datetime.now().strftime('%y%m%d') + '_' + 'learning_curve.csv' # create file name starting with yymmdd_
with open('generated_data/' + filename, 'w') as f:
    f.write('training_set_size,training_error,test_error\n')

# Generate learning curves and save them
for m in tqdm_notebook(range(settings['learning_curve']['start'], settings['learning_curve']['stop'], settings['learning_curve']['step'])):
    # Slice dataset
    X_train_red = X_train[:m]
    y_train_red = y_train[:m]
    
    # Applu linear regression
    from sklearn.linear_model import LinearRegression
    lnr_regr = LinearRegression(n_jobs=-1)
    print("Trainig for training set size of: ", m, "...")
    lnr_regr.fit(X_train_red, y_train_red)
    
    # Make predictions and compute RMSEs
    pred_train = lnr_regr.predict(X_train_red)
    rmse_train = rmse(y_train_red, pred_train)
    print("Training set RMSE: %.2f" % rmse_train)
    pred_test = lnr_regr.predict(X_test)
    rmse_test = rmse(y_test, pred_test)
    print("Training set RMSE: %.2f" % rmse_test)
    
    # Save to csv file
    print("Saving to file...")
    with open('generated_data/' + filename, 'a') as f:
        f.write(str(m) + ',' + str(rmse_train) + ',' + str(rmse_test) +'\n')

# Apply linear regression to the full trainin set, compute training and test RMSE and add them to the file created above

In [ ]:
# Fit linear regression
from sklearn.linear_model import LinearRegression
lnr_regr = LinearRegression(n_jobs=-1)
print("Trainig...")
lnr_regr.fit(X_train, y_train)
print("Training done.")

if (settings['save_env']):
    dill.dump_session('linearModel.pkl')
    
# Make predictions and report train and test RMSEs

print("Evaluating performance on the training set...")
pred_train = lnr_regr.predict(X_train)
rmse_train = rmse(y_train, pred_train)
print("Training set RMSE: %.2f" % rmse_train)

print("Evaluating performance on the test test...")
pred_test = lnr_regr.predict(X_test)
rmse_test = rmse(y_test, pred_test)
print("Test set RMSE: %.2f" % rmse_test)

# Save to csv file
print("Saving to file...")
with open('generated_data/' + filename, 'a') as f:
    f.write(str(len(X_train)) + ',' + str(rmse_train) + ',' + str(rmse_test) +'\n')

In [ ]:
# import dill

In [ ]:
# dill.load_session('linearModel.pkl')

In [ ]:
# who